In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy # Add Deepcopy for args

import seaborn as sns 
import matplotlib.pyplot as plt
import os, pickle, joblib, argparse

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.api import SimpleExpSmoothing
from sklearn.preprocessing import KBinsDiscretizer

# pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
 
print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 6)

1.6.0+cpu
Populating the interactive namespace from numpy and matplotlib


In [3]:
# # read file
# raw_x_df = pd.read_csv('./data/train_x_df.csv')
# raw_y_df = pd.read_csv('./data/train_y_df.csv')

# print('Read files Complete!')

In [4]:
# ================================================= #
def df2d_to_array3d(df_2d):
    
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    
    print('DataFrame to array, Complete!')
    
    return array_3d

# ================================================= #

def simple_exponetial_smoothing_fory(arr, alpha=0.3):
    
    y_series = list()

    for temp_arr in arr:
        target_series = temp_arr[:].reshape(-1) # open col is 1 index

        smoother = SimpleExpSmoothing(target_series, initialization_method="heuristic").fit(smoothing_level=alpha,optimized=False)
        smoothing_series = smoother.fittedvalues

        y_series.append(smoothing_series)
            
    return np.array(y_series)

# ================================================= #

def simple_exponetial_smoothing_forX(arr, alpha=0.3):
    
    # initialization
    sample_size = int(arr.shape[0])
    time_size = int(arr.shape[1])
    feature_size = int(arr.shape[2])
    
    # create empty array
    smoothing_arr = np.zeros((sample_size, time_size, feature_size - 1))

    for idx, temp_arr in enumerate(arr):
        for col in range(1, feature_size): # open col is 1 index
            if col < 5:

                temp_series = temp_arr[:, col].reshape(-1) 
                smoother = SimpleExpSmoothing(temp_series, initialization_method="heuristic").fit(smoothing_level=0.3,optimized=False)
                temp_smoothing_series = smoother.fittedvalues
                smoothing_arr[idx, :, col-1] = temp_smoothing_series

            else:
                
                pass_series = temp_arr[:, col].reshape(-1)
                smoothing_arr[idx, :, col-1] = pass_series

    return smoothing_arr


# ================================================= #
def kbindiscreter(input_array):
    kb = KBinsDiscretizer(n_bins=100, strategy='uniform', encode='ordinal')
    processed_data = np.zeros((input_array.shape[0], input_array.shape[1], 1))
    
    for i in range(input_array.shape[0]):
        # coin_index_export args : (input_array, coin_num)
        processing_array = input_array[i,:,1]
        #globals()['outliery_array{}'.format(i)] = train_y_array[outlier[i],:,1]
        kb.fit(processing_array.reshape(input_array.shape[1],1))
        processed_fit = kb.transform(processing_array.reshape(input_array.shape[1],1))
        #globals()['outliery_fit{}'.format(i)] = kb.transform(globals()['outliery_array{}'.format(i)].reshape(120,1))
        processed_data[i,:,:] = processed_fit
        
    return processed_data

def moving_average(arr, window_size = 60):
    
    #length = ma 몇 할지
    length = window_size
    ma = np.zeros((arr.shape[0], arr.shape[1] - length, arr.shape[2]))

    for idx in range(arr.shape[0]):
        for i in range(length, arr.shape[1]):
            for col in range(arr.shape[2]):
                ma[idx, i-length, col] = arr[idx,i-length:i, col].mean() #open
            
    return ma

In [116]:
merged_df = pd.read_hdf('./data/merged_allcoin.h5')

In [117]:
merged_arr = df2d_to_array3d(merged_df)

DataFrame to array, Complete!


In [118]:
merged_arr.shape

(7661, 1500, 10)

In [119]:
# with open('./data/ma60_merged_data.npy', 'rb') as f:
#     ma60_merged_arr = np.load(f) 

In [10]:
def time_split(input_array, split_size = 6):

    # origin size define
    index_size = input_array.shape[0]
    origin_time_size = input_array.shape[1]
    variable_size = input_array.shape[2]

    # new array size define
    new_time_size = int(origin_time_size/split_size) # 1380 / 6
    new_array = np.zeros((index_size, new_time_size, variable_size))

    for idx in range(index_size):
        for time_idx in range(new_time_size):
            

            first_time_idx = time_idx * split_size
            last_time_idx = ((time_idx+1) * split_size) -1

            new_array[idx, time_idx, 0] = input_array[idx, first_time_idx, 0] #coin_num
    return new_array
  

In [286]:
# bins_merged_arr = kbindiscreter(merged_arr)
# split_merged_arr = time_split(bins_merged_arr)
# mooth_merged_arr = simple_exponetial_smoothing_fory(split_merged_arr)

# bins_x_arr = kbindiscreter(raw_x_arr)
# bins_y_arr = kbindiscreter(raw_y_arr)

In [287]:
# bins_x_arr = time_split(bins_x_arr)
# bins_x_arr = simple_exponetial_smoothing_fory(bins_x_arr)

# bins_y_arr = time_split(bins_y_arr)
# bins_y_arr = simple_exponetial_smoothing_fory(bins_y_arr)

In [122]:
merged_arr.shape

(7661, 1500, 10)

In [123]:
# open_y_arr = np.zeros((7661, 5, 1))

# for i in range(7661):
#     for j in range(5):
#         open_y_arr[i, j, :] = raw_y_arr[i, 29 * j, 1]

        
# y_train = np.zeros((7661))
# for i in range(7661):
#     y_train[i, ] = np.argmax(open_y_arr[i])


In [126]:

# train & val set
train_set, val_set = train_test_split(merged_arr, test_size = 0.2, shuffle=True)


In [128]:
len(train_set), len(val_set)

(6128, 1533)

In [129]:
# # train & val set
# train_X, val_X, train_y, val_y = train_test_split(raw_x_arr[:, :, 1:], y_train, test_size = 0.1, shuffle=False)

# print(
# f'''
# ======================================================
# Origin length is {len(raw_x_arr)}, then total split length is {len(raw_x_arr)} + {len(val_X)} = {len(train_X)+len(val_X)}
# ======================================================
# train X length is {train_X.shape}, train y length is {train_y.shape},
# val X length is {val_X.shape}, val y length is {val_y.shape},
# '''
# # test X length is {test_X.shape}, test y length is {test_y.shape}
# )

In [131]:
# ====== initialization
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is",args.device)

seed = 777
np.random.seed(seed)
torch.manual_seed(seed)

# ====== Model Capacity options ===== #
args.input_dim = 9
args.hidden_dim = 64
args.output_dim = 1
args.n_layers = 1
args.batch_size = 16
args.x_frames = 1380
args.y_frames = 120

# ====== Model training options ===== #
args.num_epoch = 20
args.learning_rate = 0.0001
args.L2_rate = 0.00001


device is cpu


In [280]:
class WindowGenerator(TensorDataset):
    
    ''' Dataset Generate'''
    def __init__(self, data_set, x_frames, y_frames):
    
        self.data_set = data_set
        self.x_frames = x_frames
        self.y_frames = y_frames

    def __len__(self):
        return len(self.data_set)

    def __getitem__(self, idx):
        
        X = self.data_set[idx, :-self.y_frames, 1:]
        y = self.data_set[idx, -self.y_frames:, 1]
        
        X = np.log(X + 1) - np.log(X[-1, :] + 1)
    
        
        return X, y
    
    

In [281]:
trainset = WindowGenerator(train_set, 1380, 120) # elf, data_set, x_frames, y_frames):
trainloader = DataLoader(trainset, batch_size = args.batch_size,shuffle = True, drop_last = True)

In [285]:
for i, (X,y) in enumerate(trainloader):
    print(X.transpose(1, 2).shape)
    print(y.shape)
    break

torch.Size([16, 9, 1380])
torch.Size([16, 120])


In [288]:
class CNN_LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CNN_LSTM, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.conv1 = nn.Conv1d(arg.input_dim, args.hidden_dim, kernel_size = 10)
        self.pooling1 = nn.MaxPool1d(2, stride = 5)
        self.conv2 = nn.Conv1d(args.hidden_dim, args.hidden_dim // 2, kernel_size = 5)
        self.pooling2 = nn.MaxPool1d(2, stride = 2)
        
        self.norm = nn.BatchNorm1d(args.hidden_dim // 2)
        
        self.lstm = nn.LSTM(args.hidden_dim // 2, (args.hidden_dim // 2)/2 , 1, batch_first = True, bidirectional = True)
        self.linear = nn.Linear(args.hidden_dim // 2, args.output_dim)
        
        
    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))
    
    
    def forward(self, X):
        
        # input은 (Batch, Feature dimension, Time_step)순
        output = F.relu(conv1(X))
        output = pooling1(output)
        output = F.relu(conv2(output))
        output = pooling2(output)
        output = norm1(output)
        
        # [Batch_size, Seq_len, Hidden_size]
        print(output.shape)
        output = output.transpose(1, 2)
        print(output.shape)
        
        output, self.hidden = self.lstm(output)
        y_pred = self.linear(output[:, -1, :])
        
        return y_pred


In [171]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.dropout = dropout
        self.use_bn = use_bn 
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first =True)

        self.fc = self.make_regressor()
        
    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, 200))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(200, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, X):
        lstm_out, self.hidden = self.lstm(X)
        y_pred = self.fc(lstm_out[:, -1, :])
        return y_pred

In [174]:
def train(model, partition, optimizer, loss_fn, args):
    ''' model training '''
   
    # data load
    trainloader = DataLoader(partition['train'],
                             batch_size = args.batch_size,
                             shuffle = True, drop_last = True)
    
    # model's mode setting
    model.train()
    model.zero_grad()
    optimizer.zero_grad()
    
    train_loss = 0.0
    for i, (X, y) in enumerate(trainloader):
    
        X = X.float().to(args.device)
        y_true = y.long().to(args.device)
        
#         print(X.shape, y_true.shape)
        
        # zero the gradient
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = model.init_hidden(X.shape[1])

        y_pred = model(X)
#         print(y_pred.shape)

        loss = loss_fn(y_pred, y_true)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss = train_loss / len(trainloader)
    return model, train_loss


def validate(model, partition, loss_fn, args):
    ''' model validate '''
    
    # data load
    valloader = DataLoader(partition['val'], 
                           batch_size = args.batch_size, 
                           shuffle = False, drop_last = True)
    
    # model's mode setting
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    # evaluate
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):
            
            X = X.float().to(args.device)
            y_true = y.long().to(args.device)
            
            model.hidden = model.init_hidden(X.shape[1])
            # en-decoder outputs tensor 
            y_pred = model(X)
            
            # compute the loss
            loss = loss_fn(y_pred, y_true)
            val_loss += loss.item()
            
            preds = F.log_softmax(y_pred, dim=1).argmax(dim=1)
            total += y_true.size(0)
            correct += (preds == y_val).sum().item()
            
    val_loss = val_loss / len(valloader)
    val_acc = correct / total
    return val_loss, val_acc


In [63]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hidden_dim, args.output_dim, args.n_layers, args.dropout, args.use_bn)
    model.to(args.device)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)
    
    # epoch-wise loss
    train_losses = []
    val_losses = []
    val_accs = []
    
    for epoch in range(args.num_epoch):
        
        start_time = time.time()
        model, train_loss = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc = validate(model, partition, loss_fn, args)
        end_time = time.time()
        
        # add epoch loss
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'.format(
            epoch+1, val_acc, train_loss, val_loss, end_time-start_time))
    

    # ======= Add Result to Dictionary ======= #
    result = {}
    
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
        
    return vars(args), result, model

In [64]:
# train_X, val_X, train_y, val_y = train_test_split(raw_x_arr[:, :, 1:], y_train, test_size = 0.1, shuffle=False)
trainset = WindowGenerator(train_X, train_y, args.x_frames)
valset = WindowGenerator(val_X, val_y, args.x_frames)

partition = {'train': trainset, 'val':valset}

In [65]:
print(args)
setting, result, model = experiment(partition, deepcopy(args))

Namespace(L2_rate=0.0001, batch_size=16, device='cpu', dropout=0.2, hidden_dim=100, input_dim=9, learning_rate=0.0001, n_layers=1, num_epoch=20, output_dim=9, use_bn=True, x_frames=1380)


KeyboardInterrupt: 

In [ ]:
testloader = DataLoader(partition['train'], batch_size = 1, shuffle = True, drop_last = True)
model.eval()
# evaluate

with torch.no_grad():
    for i, (X, y) in enumerate(testloader):

        X = X.transpose(0, 1).float().to(args.device)
        y_true = y.float().to(args.device)
        model.hidden = model.init_hidden(X.shape[1])

        # en-decoder outputs tensor 
        y_pred = model(X)
        
        # y values to cpu
        y_true = y_true.cpu().detach().numpy().reshape(-1)
        y_pred = y_pred.cpu().detach().numpy().reshape(-1)
        
        print(y_true.shape, y_pred.shape)

        plt.plot(y_true, label = 'True series')
        plt.plot(y_pred, '-', label = 'Prediction series')
#         plt.plot(.max(), '*')
        plt.legend()
        plt.show()

        loss = mean_absolute_error(y_true, y_pred)
        mae =+ loss
        if i == 10:
            print(args, "\nSES 사용")
            print(f'mean absolute error * 10E5 is {(mae/5) * 10E5}')            
            break
